In [1]:
import tensorflow as tf

/Users/niikurasayaka/.conda/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/niikurasayaka/.conda/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/niikurasayaka/.conda/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/niikurasayak

# 【問題1】スクラッチを振り返る

<br>各層で重みとバイアスを作成する。
<br>・各ノードの計算結果を活性化関数を使って変換。
<br>・誤差逆伝盤法によって重みを更新
<br>・畳み込み層ではカーネルを設定し、スライドさせながら積和計算にて畳み込む
<br>・プーリング層で圧縮
<br>・最後は全結合層に繋ぐ


# 【問題2】スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。

<br>・先に定義して実行はSessionにて
<br>・宣言の中で重み、バイアスの初期化、層の構成までを全て定義してしまう。
<br>・誤差逆伝盤→重みの更新を定義したのみで順次逆に戻ってやってくれる。

# 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成

In [4]:
!pip install sklearn

     |████████████████████████████████| 7.1 MB 8.1 MB/s eta 0:00:01     |██                              | 460 kB 8.1 MB/s eta 0:00:01
     |████████████████████████████████| 294 kB 48.1 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1315 sha256=8989cb25f67cbab77044b9fbadbf59f585433e4526b7895002885a9553e2f276
  Stored in directory: /Users/niikurasayaka/Library/Caches/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [14]:
!pip install pandas

     |████████████████████████████████| 9.8 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 509 kB 37.6 MB/s eta 0:00:01


In [19]:
from sklearn.datasets import load_iris
import pdb
from keras import backend as K

In [3]:
iris = load_iris()

In [4]:
data = iris.data
feature_names = iris.feature_names
label = iris.target

In [5]:
import pandas as pd
df = pd.DataFrame(data, columns=feature_names)

In [6]:
df["target"] = label

In [7]:
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [41]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# データセットの読み込み
y = df["target"]
X = df.iloc[:, 0:4]
y = np.array(y)
y = np.reshape(y, [-1, 1])
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
y = ohe.fit_transform(y)

X = np.array(X)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)


In [43]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [44]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [45]:
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    
    
    return layer_output

In [47]:
# ネットワーク構造の読み込み                                
layer_output = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=layer_output))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

#推定結果
soft = tf.nn.softmax(layer_output)
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(soft, 1))

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 56.2574, val_loss : 28.1458, acc : 0.333, val_acc : 0.708
Epoch 1, loss : 9.9296, val_loss : 26.8956, acc : 0.833, val_acc : 0.625
Epoch 2, loss : 2.3312, val_loss : 4.2188, acc : 0.667, val_acc : 0.833
Epoch 3, loss : 0.0000, val_loss : 1.9737, acc : 1.000, val_acc : 0.917
Epoch 4, loss : 0.0000, val_loss : 2.5164, acc : 1.000, val_acc : 0.833
Epoch 5, loss : 0.0000, val_loss : 1.7732, acc : 1.000, val_acc : 0.958
Epoch 6, loss : 0.0000, val_loss : 1.4428, acc : 1.000, val_acc : 0.917
Epoch 7, loss : 0.0000, val_loss : 1.6698, acc : 1.000, val_acc : 0.875
Epoch 8, loss : 0.0000, val_loss : 1.3966, acc : 1.000, val_acc : 0.875
Epoch 9, loss : 0.0000, val_loss : 1.4422, acc : 1.000, val_acc : 0.958
test_acc : 1.000


# 【問題4】House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。

In [187]:
df = pd.read_csv("/Users/niikurasayaka/diveintocode-ml/Week3/data/train.csv")

In [188]:
df.shape

(1460, 81)

In [189]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [190]:
y = df["SalePrice"]
y = np.log(np.array(y))
y = np.reshape(y, [-1, 1])
X = df[['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea',
       '1stFlrSF', 'TotalBsmtSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt']]
X = np.array(X)

In [191]:
from sklearn.preprocessing import StandardScaler

In [192]:
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)


In [193]:
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [194]:
X_train.shape

(934, 9)

In [195]:
# ハイパーパラメータの設定
learning_rate = 0.00001
batch_size = 934
num_epochs = 2000
n_hidden1 = 200
n_hidden2 = 500
n_hidden3 = 500
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [196]:
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_hidden3])),
        'w4': tf.Variable(tf.random_normal([n_hidden3, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_hidden3])),
        'b4': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)
    layer_output = tf.matmul(layer_3, weights['w4']) + biases['b4'] # tf.addと+は等価である
    return layer_output

In [197]:
# ネットワーク構造の読み込み                                
layer_output = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.keras.losses.MSE(Y, layer_output))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(1, num_epochs+1):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss= sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss = sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
        if epoch % 50==0:
            print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}".format(epoch, loss, val_loss))
    test_loss = sess.run(loss_op, feed_dict={X: X_test, Y: y_test})
    print("test_loss : {:.3f}".format(test_loss))

Epoch 50, loss : 17146682.0000, val_loss : 17484376.0000
Epoch 100, loss : 14932162.0000, val_loss : 15585912.0000
Epoch 150, loss : 13259632.0000, val_loss : 14133313.0000
Epoch 200, loss : 11892141.0000, val_loss : 12905220.0000
Epoch 250, loss : 10720571.0000, val_loss : 11813190.0000
Epoch 300, loss : 9687627.0000, val_loss : 10822536.0000
Epoch 350, loss : 8765386.0000, val_loss : 9917568.0000
Epoch 400, loss : 7939779.0000, val_loss : 9088768.0000
Epoch 450, loss : 7197534.5000, val_loss : 8332608.0000
Epoch 500, loss : 6530650.5000, val_loss : 7643946.5000
Epoch 550, loss : 5931131.5000, val_loss : 7016434.5000
Epoch 600, loss : 5391642.0000, val_loss : 6446921.5000
Epoch 650, loss : 4906164.5000, val_loss : 5930939.0000
Epoch 700, loss : 4470330.5000, val_loss : 5465206.5000
Epoch 750, loss : 4078997.7500, val_loss : 5045517.0000
Epoch 800, loss : 3727767.0000, val_loss : 4667009.5000
Epoch 850, loss : 3413143.0000, val_loss : 4325710.0000
Epoch 900, loss : 3130908.2500, val_lo

# 【問題5】MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。

In [169]:
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [170]:
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
print(X_train.max()) # 1.0
print(X_train.min()) 

1.0
0.0


In [171]:
X_train = np.reshape(X_train, [-1, 784])
X_test = np.reshape(X_test, [-1, 784])

In [172]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])
print(y_train.shape) # (60000,)
print(y_train_one_hot.shape) # (60000, 10)
print(y_train_one_hot.dtype) # float64

(60000,)
(60000, 10)
float64


In [173]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2)
print(X_train.shape) # (48000, 784)
print(X_val.shape) # (12000, 784)

(48000, 784)
(12000, 784)


In [180]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [181]:
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    
    
    return layer_output

In [183]:
# ネットワーク構造の読み込み                                
layer_output = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=layer_output))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

#推定結果
soft = tf.nn.softmax(layer_output)
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(soft, 1))

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 0.6149, val_loss : 1.0750, acc : 0.800, val_acc : 0.771
Epoch 1, loss : 0.6519, val_loss : 0.7123, acc : 0.800, val_acc : 0.820
Epoch 2, loss : 0.2376, val_loss : 0.5395, acc : 1.000, val_acc : 0.874
Epoch 3, loss : 0.2119, val_loss : 0.3701, acc : 0.900, val_acc : 0.904
Epoch 4, loss : 0.5973, val_loss : 0.3926, acc : 0.700, val_acc : 0.907
Epoch 5, loss : 0.0398, val_loss : 0.3462, acc : 1.000, val_acc : 0.915
Epoch 6, loss : 0.0952, val_loss : 0.3581, acc : 1.000, val_acc : 0.915
Epoch 7, loss : 0.1402, val_loss : 0.3647, acc : 1.000, val_acc : 0.914
Epoch 8, loss : 0.0979, val_loss : 0.4027, acc : 1.000, val_acc : 0.914
Epoch 9, loss : 0.0439, val_loss : 0.4979, acc : 1.000, val_acc : 0.915
test_acc : 0.916
